A notebook to run stage 3 analysis of the P1490 dataset

In this script we load the first-level GLMs for each ppt, and compute second level results. We will then compute a group average.

1. Load in first-level GLM per ppt
2. Compute group-level responses for all ROIs
3. View group-beta maps per condition OR contrast conditions for V1 and LGN
4. Statistical analysis
    - ANOVA per ROI
    - post-hoc t-tests comparing conditions
    - t-tests > 0
5. Plot these results as violin plots

In [ ]:
# Import necessary libraries
import statsmodels.api as sm
from statsmodels.stats.anova import AnovaRM
import pingouin as pg
import os
from joblib import dump, load
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from nilearn import image
from nilearn.reporting import make_glm_report
from nilearn.image import load_img, threshold_img
from nilearn.glm.contrasts import compute_contrast
from nilearn.input_data import NiftiMasker
from nilearn.plotting import view_img
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.second_level import SecondLevelModel
from nilearn.maskers import NiftiMasker, NiftiLabelsMasker
from nilearn.plotting import view_img_on_surf
import pandas as pd
from collections import defaultdict
import json, joblib
import pandas as pd
from nilearn.image import mean_img
from nilearn.glm.first_level import FirstLevelModel
from nilearn.masking import apply_mask
import seaborn as sns

print('Done importing libraries')

In [ ]:
# Set up

RECOMPUTE_DATA = False

# subjects
all_pptIDs = sorted(['R5619','R6068','R6159','R6376','R6380','R6425','R6502','R6591','R6611','R6619','R6669','R6688','R6694','R6702','R6770','R6804'])
nPpt = len(all_pptIDs)

# directories
base_dir = '/scratch/groups/Projects/P1490'
data_dir = 'Data' # directory of all participants' data
t1_dir='freesurfer' # directory where recon-all T1 is
atlas_path = os.path.join(base_dir,t1_dir,'MNI/mri/tempBens/benson14_eccen_4_10_varea.nii.gz')
lgn_path = os.path.join(base_dir,data_dir,'LGN_atlas/9-LGN.nii.gz')
group_dir = os.path.join(base_dir, data_dir, "group_betas")

# condition contrasts
conditions = ['LumDiskIn','LumDiskOut','LMDiskIn','LMDiskOut','SconeDiskIn','SconeDiskOut','LumGratingIn','LumGratingOut','LMGratingIn','LMGratingOut','SconeGratingIn','SconeGratingOut','ITI']
# we have to use Scone rather than S - S includes 'Disk'

# atlas
atlas = load_img(atlas_path) # V1
atlas_data  = atlas.get_fdata()

lgn_atlas = load_img(lgn_path)
lgn_atlas_data  = lgn_atlas.get_fdata()

roi_names = ['V1','V2','V3','hV4','VO1','VO2','V3B','V3A','LO1','LO2','TO1','TO2'] # 1-12, 0=background
nROI = np.arange(len(roi_names))
label2name  = {l: f"ROI{l}" for l in roi_names}

roi_labels = np.unique(atlas_data.astype(int)) 
roi_labels = roi_labels[roi_labels != 0]


In [ ]:
if RECOMPUTE_DATA:
    # PER-SUBJECT LOOP 
    subj_beta_imgs = defaultdict(list)   # for later group maps
    # save V1 and LGN beta values
    all_betas = []             
    lgn_all_betas = []

    for p, pptID in enumerate(all_pptIDs):
        print(f"\n[{p+1}/{nPpt}] Processing subject {pptID}")

        subject_betas = []
        lgn_subject_betas = []

        # load in the preprocessed, within ppt GLM data
        models = joblib.load(os.path.join(base_dir, data_dir, pptID, "L1_fits.joblib"))
        
        # ---------- condition regressors ----------
        # the first 13 columns are the ones we want (conditions + ISI), so we can ignore the last 7
        run_cols = [[str(cond) for cond in model.design_matrices_[0].columns[:13]]
                    for model in models]
        condition_cols = set(run_cols[0]).intersection(*run_cols[1:])
        condition_cols_sorted = sorted(condition_cols, key=lambda x: int(x))
        print("     condition columns:", condition_cols_sorted)

        # ---------- extract beta maps (per run) ----------
        # this creates a dictionary with CONDITION: beta map x5
        run_betas = {cond: [] for cond in condition_cols_sorted}

        for model in models:
            cols = [str(cond) for cond in model.design_matrices_[0].columns]
            for cond in condition_cols_sorted:
                vec = np.zeros(len(cols))    
                vec[cols.index(cond)] = 1.0
                beta_img = model.compute_contrast(vec, output_type="effect_size")
                run_betas[cond].append(beta_img)

        # ---------- average runs - one beta map per regressor ----------
        subj_betas = {cond: mean_img(imgs) for cond, imgs in run_betas.items()}
        print('beta maps extracted')

        # ---------- V1 ----------
        # resample atlas to the beta map geometry 
        ref_img = next(iter(subj_betas.values()))   # use the first beta map as reference
        atlas_resampled = image.resample_to_img(atlas, ref_img, interpolation='nearest', force_resample=True)
        atlas_resampled_data = np.squeeze(atlas_resampled.get_fdata().astype(int))

        # extract ROI means for every condition 
        sub_dir = os.path.join(base_dir, data_dir, pptID,'raw_betas')
        os.makedirs(sub_dir, exist_ok=True)

        for cond, img in subj_betas.items():

            beta_data = img.get_fdata()

            for roi in roi_labels:

                mask = (atlas_resampled_data == roi)
                if not mask.any(): # label absent in volume
                    continue   
                this_data = beta_data[mask]

                # here we calculate the mean and remove any voxels that are greater than 2SD away from the mean
                mean = this_data.mean()           
                std  = this_data.std(ddof=1)    
                mask_std = np.abs(this_data - mean) <= 2 * std  # to remove extreme voxels eg blood vessels or outside the mask
                cleaned_data = this_data[mask_std]

                beta_value = cleaned_data.mean() # average values in this region
                beta_dict = dict(subject=pptID, condition=str(cond), roi=roi, beta=beta_value)

                subject_betas.append(beta_dict)
                all_betas.append(beta_dict)
            
                np.savetxt(os.path.join(base_dir, data_dir, pptID, f'raw_betas/voxels/voxels_cond{cond}_roi{roi}_all.npy'), this_data, delimiter=',')
                np.savetxt(os.path.join(base_dir, data_dir, pptID, f'raw_betas/voxels/voxels_cond{cond}_roi{roi}_clean.npy'), cleaned_data, delimiter=',')                

        # ---------- LGN ----------
        # resample atlas to the beta map geometry 
        ref_img = next(iter(subj_betas.values()))   # use the first beta map as reference
        atlas_resampled = image.resample_to_img(lgn_atlas, ref_img, interpolation='nearest', force_resample=True)
        atlas_resampled_data = np.squeeze(atlas_resampled.get_fdata().astype(int))

        # extract ROI means for every condition 
        sub_dir = os.path.join(base_dir, data_dir, pptID,'lgn_raw_betas')
        os.makedirs(sub_dir, exist_ok=True)

        for cond, img in subj_betas.items():

            beta_data = img.get_fdata()

            mask = (atlas_resampled_data == 1)
            if not mask.any(): # label absent in volume
                continue   
            this_data = beta_data[mask]

            # here we calculate the mean and remove any voxels that are greater than 2SD away from the mean
            mean = this_data.mean()           
            std  = this_data.std(ddof=1)    
            mask_std = np.abs(this_data - mean) <= 2 * std  # to remove extreme voxels eg blood vessels or outside the mask
            cleaned_data = this_data[mask_std]

            beta_value = cleaned_data.mean() # average values in this region
            beta_dict = dict(subject=pptID, condition=str(cond), roi='LGN', beta=beta_value)

            lgn_subject_betas.append(beta_dict)
            lgn_all_betas.append(beta_dict)
        
            np.savetxt(os.path.join(base_dir, data_dir, pptID, f'raw_betas/voxels/voxels_cond{cond}_roiLGN_all.npy'), this_data, delimiter=',')
            np.savetxt(os.path.join(base_dir, data_dir, pptID, f'raw_betas/voxels/voxels_cond{cond}_roiLGN_clean.npy'), cleaned_data, delimiter=',')        

        print('LGN extracted')

        # ---------- save ----------
        for cond, img in subj_betas.items():
            nib.save(img, os.path.join(sub_dir, f"{int(cond):03d}_beta.nii.gz"))
            subj_beta_imgs[cond].append(img)

        df = pd.DataFrame(subject_betas)
        df.to_csv(os.path.join(sub_dir, "roi_beta_long.csv"), index=False)
        df_lgn = pd.DataFrame(lgn_subject_betas)
        df_lgn.to_csv(os.path.join(sub_dir, "lgn_beta_long.csv"), index=False)
        print("Saved:", pptID)

    # GROUP-LEVEL AVERAGE
    # maps 
    for cond, imgs in subj_beta_imgs.items():
        group_img = mean_img(imgs)
        nib.save(group_img, os.path.join(group_dir, f"{int(cond):03d}_beta_mean.nii.gz"))

    # beta values
    df = pd.DataFrame(all_betas)
    df.to_csv(os.path.join(group_dir, "roi_beta_long.csv"), index=False)
    df_lgn = pd.DataFrame(lgn_all_betas)
    df_lgn.to_csv(os.path.join(group_dir, "lgn_beta_long.csv"), index=False)
    print("Saved: group")
else:
    print(f'Loading predefined data')
    # Load the csv file into a dataframe
    dtype_map = {'subject': 'category', 'condition': 'category', 'roi': 'category'}
    df = pd.read_csv(os.path.join(group_dir,"roi_beta_long.csv"), dtype=dtype_map)
    df_lgn = pd.read_csv(os.path.join(group_dir,"lgn_beta_long.csv"), dtype=dtype_map)

In [ ]:
# voxel histograms
# just to make sure there are no odd or extreme voxels (eg blood vessels, outside mask etc.)
# generally pretty good

for pptID in all_pptIDs:

    lgn_roi_labels = list(map(str, roi_labels)) + ['LGN']

    for roi in lgn_roi_labels:

        plt.close()
        plt.figure(figsize=(8,12))

        for cond in np.arange(1, len(conditions)+1):

            n_voxels_rm = 0
            total_voxels = 0 

            if cond == 13:
                i = 100
            else:
                i = cond

            voxels_all = np.loadtxt(os.path.join(base_dir, data_dir, pptID, f'raw_betas/voxels/voxels_cond{i}_roi{roi}_all.npy'), delimiter=',')
            voxels_clean = np.loadtxt(os.path.join(base_dir, data_dir, pptID, f'raw_betas/voxels/voxels_cond{i}_roi{roi}_clean.npy'), delimiter=',')

            voxels_removed = np.size(voxels_all) - np.size(voxels_clean)
            n_voxels_rm = n_voxels_rm + voxels_removed
            total_voxels = total_voxels + np.size(voxels_all)

            np.savetxt(os.path.join(base_dir, data_dir, pptID, f"raw_betas/voxels/n_voxels_removed_roi{roi}_cond{i}.csv"), [n_voxels_rm], fmt='%s')
            np.savetxt(os.path.join(base_dir, data_dir, pptID, f"raw_betas/voxels/n_voxels_total_roi{roi}_cond{i}.csv"), [total_voxels], fmt='%s')

            plt.subplot(3,6,cond)

            bin_width = 0.25
            data_min = min(voxels_all.min(), voxels_clean.min())
            data_max = max(voxels_all.max(), voxels_clean.max())
            bins = np.arange(data_min, data_max + bin_width, bin_width)

            plt.hist(voxels_all, bins=bins, alpha=0.5, color='blue', label='all')
            plt.hist(voxels_clean, bins=bins, alpha=0.5, color='red', label='clean')
            plt.title(f'{cond}')
            plt.xlabel('Value')
            plt.ylabel('Frequency')
        plt.tight_layout()
        plt.savefig(os.path.join(base_dir, data_dir, pptID, f'raw_betas/voxels/voxels_hists_roi{roi}.png'))
        #plt.show()

        plt.close()


In [ ]:
# We can view the average beta maps for a particular condition in 3D volume
# the average response is the change from baseline (ITI)
cond = 12

mean_beta_map = nib.load(os.path.join(group_dir, f"{cond:03d}_beta_mean.nii.gz"))
iti_beta_map = nib.load(os.path.join(group_dir, "100_beta_mean.nii.gz"))

mean_beta_data = mean_beta_map.get_fdata()
iti_beta_data = iti_beta_map.get_fdata()

change_beta_data = mean_beta_data - iti_beta_data

change_beta_map = nib.Nifti1Image(change_beta_data, mean_beta_map.affine, mean_beta_map.header)
nib.save(change_beta_map, os.path.join(group_dir, f'{cond:03d}_beta_change.nii.gz'))

view=view_img(change_beta_map, black_bg=True, threshold=0.2)
view

In [ ]:
# apply V1 benson atlas
# Calculate the average voxel response in that region (mean and median - but very little difference)
# and view the response on 3D volume

cond = 3

change_beta_map = nib.load(os.path.join(group_dir, f'{cond:03d}_beta_change.nii.gz'))
change_beta_data = change_beta_map.get_fdata()

atlas_resampled = image.resample_to_img(atlas, change_beta_map, interpolation='nearest', force_resample=True)
atlas_resampled_data = np.squeeze(atlas_resampled.get_fdata().astype(int))

#mask = np.isin(atlas_resampled_data, roi_labels)           
mask = (atlas_resampled_data == 1) 

masked_data = np.where(mask, change_beta_data, np.nan)   

mean_beta = np.nanmean(masked_data)
print(f"Region: V1 — Mean beta: {mean_beta:.4f}")
med_beta = np.nanmedian(masked_data)
print(f"Region: V1 — Med beta: {med_beta:.4f}")

masked_map = nib.Nifti1Image(masked_data, change_beta_map.affine, change_beta_map.header)

view=view_img(masked_map, black_bg=True, threshold=0,draw_cross=False)
view

In [ ]:
# same as above for LGN

cond = 3

change_beta_map = nib.load(os.path.join(group_dir, f'{cond:03d}_beta_change.nii.gz'))
change_beta_data = change_beta_map.get_fdata()

atlas_resampled = image.resample_to_img(lgn_atlas, change_beta_map, interpolation='nearest', force_resample=True)
atlas_resampled_data = np.squeeze(atlas_resampled.get_fdata().astype(int))

mask = (atlas_resampled_data == 1) 

masked_data = np.where(mask, change_beta_data, np.nan)   

mean_beta = np.nanmean(masked_data)
print(f"Region: LGN — Mean beta: {mean_beta:.4f}")
med_beta = np.nanmedian(masked_data)
print(f"Region: LGN — Med beta: {med_beta:.4f}")

masked_map = nib.Nifti1Image(masked_data, change_beta_map.affine, change_beta_map.header)

view=view_img(masked_map, black_bg=True, threshold=0,draw_cross=False)
view

In [ ]:
# plot contrasts
# These are mainly used for visualisation in the thesis, not statistical analysis

conditions = ['LumDiskIn','LumDiskOut','LMDiskIn','LMDiskOut','SconeDiskIn','SconeDiskOut','LumGratingIn','LumGratingOut','LMGratingIn','LMGratingOut','SconeGratingIn','SconeGratingOut','ITI']
condition_numbers = np.arange(1,13)
contrast_maps = [nib.load(os.path.join(group_dir, f'{c:03d}_beta_change.nii.gz')) for c in condition_numbers]

# Define filters
include_keyword = 'DiskIn' # or None for all conditions
c1 = ['Lum']
c2 = ['LM','Scone']

c1name = ''.join(c1)
c2name = ''.join(c2)

# group coding
group_coding = []
for cond in conditions[:12]:  # First 12 conditions only
    if include_keyword is None or include_keyword in cond:
        if any(k in cond for k in c1):
            group_coding.append(1)
        elif any(k in cond for k in c2):
            group_coding.append(-1)
        else:
            group_coding.append(0)
    else:
        group_coding.append(0)

# Load all 12 beta maps
condition_numbers = np.arange(1, 13)
contrast_maps = [nib.load(os.path.join(group_dir, f'{c:03d}_beta_change.nii.gz')) for c in condition_numbers]

# Build design matrix
design_matrix = pd.DataFrame({'intercept': 1, 'group_diff': group_coding})

# Fit the model
second_level_model = SecondLevelModel()
second_level_model = second_level_model.fit(contrast_maps, design_matrix=design_matrix)

# Compute contrast: test 'group_diff' effect only
beta_contrast_map = second_level_model.compute_contrast([0, 1])

view = view_img(beta_contrast_map, threshold=2)
if include_keyword:
    view.save_as_html(os.path.join(group_dir,'contrast_maps', f'{c1name}_v_{c2name}_for{include_keyword}_volume.html'))
else:
    view.save_as_html(os.path.join(group_dir,'contrast_maps', f'{c1name}_v_{c2name}_volume.html'))
view


In [ ]:
# also display on a surface
view = view_img_on_surf(beta_contrast_map,threshold=2, surf_mesh='fsaverage7',black_bg=True, bg_on_data=True,
        vol_to_surf_kwargs=dict(
        radius=3.0,              # neighbourhood (mm)
        interpolation="linear", # 'linear' or 'nearest'
        kind="ball",             # 'auto', 'line', 'ball', or 'depth'
        n_samples=10             # number of samples to average
    )) # Change fsaverage5 to fsaverage if you want to use the higher-res fsaverage mesh
if include_keyword:
    view.save_as_html(os.path.join(group_dir,'contrast_maps', f'{c1name}_v_{c2name}_for{include_keyword}_surface.html'))
else:
    view.save_as_html(os.path.join(group_dir,'contrast_maps', f'{c1name}_v_{c2name}_surface.html'))
view 

In [ ]:
# prepare the data for statistical analysis

# keys as strings (important for merges that follow)
df["subject"]   = df["subject"].astype(str)
df["condition"] = df["condition"].astype(str)
df["roi"]       = df["roi"].astype(str)

# Isolate each subject’s condition 100 (ITI) value and merge it back
baseline = (
    df.loc[df["condition"] == "100", ["subject", "roi", "beta"]]
      .rename(columns={"beta": "beta_100"})
)

# merge - a new column with the beta 100 value on every row
df_final = df.merge(baseline, on=["subject", "roi"], how="left")

# Compute the difference (beta - beta 100) 
df_final["delta_beta"] = df_final["beta"] - df_final["beta_100"]

df_final.to_csv(os.path.join(group_dir,"roi_beta_minus_100.csv"), index=False) 

print(df_final)



# and for LGN
# keys as strings (important for merges that follow)
df_lgn["subject"]   = df_lgn["subject"].astype(str)
df_lgn["condition"] = df_lgn["condition"].astype(str)
df_lgn["roi"]       = df_lgn["roi"].astype(str)

# Isolate each subject’s condition 100 (ITI) value and merge it back
baseline = (
    df_lgn.loc[df_lgn["condition"] == "100", ["subject", "roi", "beta"]]
      .rename(columns={"beta": "beta_100"})
)

# merge - a new column with the beta 100 value on every row
df_final_lgn = df_lgn.merge(baseline, on=["subject", "roi"], how="left")

# Compute the difference (beta - beta 100) 
df_final_lgn["delta_beta"] = df_final_lgn["beta"] - df_final_lgn["beta_100"]

df_final_lgn.to_csv(os.path.join(group_dir,"lgn_beta_minus_100.csv"), index=False) # I'm removing the ISI response from every other response on a per-subject basis. This just cleans things up.

print(df_final_lgn)


In [ ]:
# ANOVA

from statsmodels.stats.anova import AnovaRM

aov = AnovaRM(data = df_final,
              depvar = 'delta_beta',
              subject = 'subject',
              within = ['condition', 'roi']).fit()
print('delta beta')
print(aov)

In [ ]:
# three way ANOVA for each roi

df_final['condition'] = df_final['condition'].astype(int)
df_with_factors = df_final[df_final['condition'] != 100].copy()

df_final_lgn['condition'] = df_final_lgn['condition'].astype(int)
df_with_factors_lgn = df_final_lgn[df_final_lgn['condition'] != 100].copy()

condition_map = {
    1:  'LumDiskIn',
    2:  'LumDiskOut',
    3:  'LMDiskIn',
    4:  'LMDiskOut',
    5:  'SconeDiskIn',
    6:  'SconeDiskOut',
    7:  'LumGratingIn',
    8:  'LumGratingOut',
    9:  'LMGratingIn',
    10: 'LMGratingOut',
    11: 'SconeGratingIn',
    12: 'SconeGratingOut',
}

df_with_factors['condition_label'] = df_with_factors['condition'].map(condition_map)
df_with_factors[['colour', 'spatial_freq', 'phase']] = df_with_factors['condition_label'].str.extract(r'(Lum|LM|Scone)(Disk|Grating)(In|Out)')

df_with_factors_lgn['condition_label'] = df_with_factors_lgn['condition'].map(condition_map)
df_with_factors_lgn[['colour', 'spatial_freq', 'phase']] = df_with_factors_lgn['condition_label'].str.extract(r'(Lum|LM|Scone)(Disk|Grating)(In|Out)')

df_with_factors['roi'] = df_with_factors['roi'].astype(int)

rois = [1, 2, 3, 4]
for r in rois:
    roi_data = df_with_factors[df_with_factors['roi'] == r]
    aov = AnovaRM(data=roi_data,
                    depvar='delta_beta',
                    subject='subject',
                    within=['colour', 'spatial_freq', 'phase']).fit()

    print(f"\n=== ROI {roi_names[r-1]} ===")
    print(aov.summary())

    if r == 1:
        post_hoc = pg.pairwise_tests(dv='delta_beta', within='condition', subject='subject', 
                                    data=roi_data, padjust='fdr_bh',effsize='cohen')
        print(post_hoc)
        
        ttest_results = []
        # Loop through each condition
        for condition, group in roi_data.groupby('condition'):
            ttest = pg.ttest(group['delta_beta'], 0, alternative='greater')  # One-sample, one-tailed
            ttest['condition'] = condition
            ttest_results.append(ttest)
        # Combine all results into one DataFrame
        results_df = pd.concat(ttest_results).reset_index(drop=True)
        print(results_df[['condition', 'T', 'p-val', 'dof', 'CI95%', 'power']])



aov = AnovaRM(data=df_with_factors_lgn,
                depvar='delta_beta',
                subject='subject',
                within=['colour', 'spatial_freq', 'phase']).fit()

print(f"\n=== ROI LGN ===")
print(aov.summary())

post_hoc_lgn = pg.pairwise_tests(dv='delta_beta', within='condition', subject='subject', 
                            data=df_with_factors_lgn, padjust='fdr_bh',effsize='cohen')
print(post_hoc_lgn)

ttest_results_lgn = []
# Loop through each condition
for condition, group in df_with_factors_lgn.groupby('condition'):
    ttest = pg.ttest(group['delta_beta'], 0, alternative='greater')  # One-sample, one-tailed
    ttest['condition'] = condition
    ttest_results_lgn.append(ttest)
# Combine all results into one DataFrame
results_df_lgn = pd.concat(ttest_results_lgn).reset_index(drop=True)
print(results_df_lgn[['condition', 'T', 'p-val', 'dof', 'CI95%', 'power']])


In [ ]:
# Calculate mean and SEM across subjects

df_final["condition"] = df_final["condition"].astype(int)
df_final["roi"]       = df_final["roi"].astype(int)

grouped_beta = df_final.groupby(['condition', 'roi'])['beta']
ave_beta = grouped_beta.agg(['mean', lambda x: x.sem()])
ave_beta = ave_beta.rename(columns={'<lambda_0>': 'sem'}).reset_index()
ave_beta = ave_beta.sort_values(by=['condition', 'roi']).reset_index(drop=True)

grouped_delta = df_final.groupby(['condition', 'roi'])['delta_beta']
ave_delta = grouped_delta.agg(['mean', lambda x: x.sem()])
ave_delta = ave_delta.rename(columns={'<lambda_0>': 'sem'}).reset_index()
ave_delta = ave_delta.sort_values(by=['condition', 'roi']).reset_index(drop=True)


In [ ]:
from statannotations.Annotator import Annotator  

dv = 'delta_beta'

# violin plots
plot_data = df_with_factors[df_with_factors["condition"] != 100]
roi_rows = plot_data[plot_data["roi"] == 1]

roi_data = df_with_factors[df_with_factors['roi'] == 1]

post_hoc = pg.pairwise_tests(dv='delta_beta', within='condition', subject='subject', 
                                    data=roi_data, padjust='fdr_bh',effsize='cohen')

# per chromaticity
lum_rows = roi_rows[roi_rows['colour'] == 'Lum']
lm_rows = roi_rows[roi_rows['colour'] == 'LM']
s_rows = roi_rows[roi_rows['colour'] == 'Scone']

# plotting here
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(9, 5),sharey=True)

# LUMINANCE
palette = sns.color_palette('Greys', n_colors=2); palette.reverse()
sns.violinplot(lum_rows, x='condition', y=dv, ax=ax1, 
               hue='phase', split=True, inner='point',palette=palette)
ax1.set_xticks([0.5, 2.5]); ax1.set_xticklabels(['Disk','Grating'])
ax1.set_xlabel(" ")
ax1.set_ylabel("Change in beta")
ax1.axhline(0,color='grey', linestyle='--', linewidth=1)
ax1.legend(title='Phase',loc='lower right')
ax1.set_title('Luminance')
ax1.spines['top'].set_visible(False); ax1.spines['right'].set_visible(False)
ax1.set_ylim([-6, 6])

#pairs = list(lum_post_hoc[['A', 'B']].itertuples(index=False, name=None))
# We want particular comparisons e.g. disk in phase compared to grating out of phase doesnt really mean anything
pairs = [(1, 2), (1, 7), (2, 8), (7, 8)] 
pvals = list(post_hoc['p-corr'])
pvals_wanted = [pvals[i] for i in [0,5,16,51]]
annotator = Annotator(ax1, pairs=pairs, data=lum_rows, x='condition', y=dv, verbose=0)
annotator.configure(test=None, text_format='star', loc='inside', verbose=0)
annotator.set_pvalues_and_annotate(pvals_wanted)

# L-M
palette = sns.color_palette('Reds', n_colors=2); palette.reverse()
sns.violinplot(lm_rows, x='condition', y=dv, ax=ax2,
               hue='phase', split=True, inner='point',palette=palette)
ax2.set_xticks([0.5, 2.5]); ax2.set_xticklabels(['Disk','Grating'])
ax2.set_xlabel("Spatial Frequency")
ax2.axhline(0,color='grey', linestyle='--', linewidth=1)
ax2.legend(title='Phase',loc='lower right')
ax2.set_title('L-M')
ax2.spines['top'].set_visible(False); ax2.spines['right'].set_visible(False)
ax2.set_ylim([-6, 6])

#pairs = list(lm_post_hoc[['A', 'B']].itertuples(index=False, name=None))
pairs = [(3, 4), (3, 9), (4, 10), (9, 10)]
pvals = list(post_hoc['p-corr'])
pvals_wanted = [pvals[i] for i in [21,26,35,60]]
annotator = Annotator(ax2, pairs=pairs, data=lm_rows, x='condition', y=dv, verbose=0)
annotator.configure(test=None, text_format='star', loc='inside', verbose=0)
annotator.set_pvalues_and_annotate(pvals_wanted)

# S
palette = sns.color_palette('Blues', n_colors=2); palette.reverse()
sns.violinplot(s_rows, x='condition', y=dv, ax=ax3,
               hue='phase', split=True, inner='point',palette=palette)
ax3.set_xticks([0.5, 2.5]); ax3.set_xticklabels(['Disk','Grating'])
ax3.set_xlabel(" ")
ax3.axhline(0,color='grey', linestyle='--', linewidth=1)
ax3.legend(title='Phase',loc='lower right')
ax3.set_title('S-cone')
ax3.spines['top'].set_visible(False); ax3.spines['right'].set_visible(False)
ax3.set_ylim([-6, 6])

#pairs = list(s_post_hoc[['A', 'B']].itertuples(index=False, name=None))
pairs = [(5, 6), (5, 11), (6, 12), (11, 12)]
pvals = list(post_hoc['p-corr'])
pvals_wanted = [pvals[i] for i in [38,43,50,65]]
annotator = Annotator(ax3, pairs=pairs, data=s_rows, x='condition', y=dv, verbose=0)
annotator.configure(test=None, text_format='star',loc='inside', verbose=0)
annotator.set_pvalues_and_annotate(pvals_wanted)

plt.tight_layout()
plt.savefig(os.path.join(base_dir,'graphs/V1_violin_2mm'))
plt.show()



In [ ]:
from statannotations.Annotator import Annotator  

dv = 'delta_beta'

# violin plots
plot_data = df_with_factors_lgn[df_with_factors_lgn["condition"] != 100]
roi_rows = plot_data

# per chromaticity
lum_rows = roi_rows[roi_rows['colour'] == 'Lum']
lm_rows = roi_rows[roi_rows['colour'] == 'LM']
s_rows = roi_rows[roi_rows['colour'] == 'Scone']

# plotting here
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(9, 5),sharey=True)

# LUMINANCE
palette = sns.color_palette('Greys', n_colors=2); palette.reverse()
sns.violinplot(lum_rows, x='condition', y=dv, ax=ax1, 
               hue='phase', split=True, inner='point',palette=palette)
ax1.set_xticks([0.5, 2.5]); ax1.set_xticklabels(['Disk','Grating'])
ax1.set_xlabel(" ")
ax1.set_ylabel("Change in beta")
ax1.axhline(0,color='grey', linestyle='--', linewidth=1)
ax1.legend(title='Phase',loc='lower right')
ax1.set_title('Luminance')
ax1.spines['top'].set_visible(False); ax1.spines['right'].set_visible(False)

# L-M
palette = sns.color_palette('Reds', n_colors=2); palette.reverse()
sns.violinplot(lm_rows, x='condition', y=dv, ax=ax2,
               hue='phase', split=True, inner='point',palette=palette)
ax2.set_xticks([0.5, 2.5]); ax2.set_xticklabels(['Disk','Grating'])
ax2.set_xlabel("Spatial Frequency")
ax2.axhline(0,color='grey', linestyle='--', linewidth=1)
ax2.legend(title='Phase',loc='lower right')
ax2.set_title('L-M')
ax2.spines['top'].set_visible(False); ax2.spines['right'].set_visible(False)

# S
palette = sns.color_palette('Blues', n_colors=2); palette.reverse()
sns.violinplot(s_rows, x='condition', y=dv, ax=ax3,
               hue='phase', split=True, inner='point',palette=palette)
ax3.set_xticks([0.5, 2.5]); ax3.set_xticklabels(['Disk','Grating'])
ax3.set_xlabel(" ")
ax3.axhline(0,color='grey', linestyle='--', linewidth=1)
ax3.legend(title='Phase',loc='lower right')
ax3.set_title('S-cone')
ax3.spines['top'].set_visible(False); ax3.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig(os.path.join(base_dir,'graphs/LGN_violin_2mm'))
plt.show()